### Import libraries

In [4]:
# Import packages
import argparse
import os
import warnings
import rasterio as rio
import pandas as pd
import fiona
import matplotlib.pyplot as plt
import numpy.ma as ma
import numpy as np
import xarray as xr
import rioxarray as rxr
from shapely.geometry import mapping, box
import geopandas as gpd
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
import math
from osgeo import gdal
import hvplot.xarray
import hvplot.pandas
import holoviews as hv
import rasterio as rio
from emit_tools import emit_xarray
import sys
warnings.simplefilter('ignore')

In [5]:
# Import emit_tools
sys.path.append(r'C:\Users\User\Documents\Python Scripts_2\EMIT\modules\modules')

### Open the .nc file

In [6]:
# Specify the file path
emit_fp = r'F:\emit\EMIT\March_s3\EMIT_L2A_RFL_001_20240308T070913_2406805_011.nc'

# open emit scene and orthoretify it.
emit_ds = emit_xarray(emit_fp,ortho=True)
emit_ds


<xarray.Dataset> Size: 5GB
Dimensions:           (latitude: 2036, longitude: 2114, wavelengths: 285)
Coordinates:
  * wavelengths       (wavelengths) float32 1kB 381.0 388.4 ... 2.493e+03
    fwhm              (wavelengths) float32 1kB ...
    good_wavelengths  (wavelengths) float32 1kB ...
  * latitude          (latitude) float64 16kB -30.06 -30.06 ... -31.16 -31.16
  * longitude         (longitude) float64 17kB 28.82 28.82 28.82 ... 29.96 29.97
    elev              (latitude, longitude) float32 17MB nan nan nan ... nan nan
    spatial_ref       int32 4B 0
Data variables:
    reflectance       (latitude, longitude, wavelengths) float32 5GB nan ... nan
Attributes: (12/40)
    ncei_template_version:             NCEI_NetCDF_Swath_Template_v2.0
    summary:                           The Earth Surface Mineral Dust Source ...
    keywords:                          Imaging Spectroscopy, minerals, EMIT, ...
    Conventions:                       CF-1.63
    sensor:                            EMIT (Earth Surface Mineral Dust Sourc...
    instrument:                        EMIT
    ...                                ...
    spatial_ref:                       GEOGCS["WGS 84",DATUM["WGS_1984",SPHER...
    geotransform:                      [ 2.88193822e+01  5.42232520e-04 -0.00...
    day_night_flag:                    Day
    title:                             EMIT L2A Estimated Surface Reflectance...
    granule_id:                        EMIT_L2A_RFL_001_20240308T070913_24068...
    Orthorectified:                    True

In [7]:
# Mask invalid reflectance data
emit_ds['reflectance'].data[:,:,emit_ds['good_wavelengths'].data==0] = np.nan


### Save as a true colour geotiff

In [8]:
# Extract the RGB channels (assuming 'wavelengths' is a coordinate in the dataset)
rgb_wavelengths = [650, 560, 470]
emit_rgb = emit_ds.sel(wavelengths=rgb_wavelengths, method='nearest')

# Get the array data representing reflectance for RGB
reflectance_array = emit_rgb['reflectance'].values

# Define the output path
output_path = r'F:\emit\EMIT\March_s3\EMIT_L2A_RFL_001_20240308T070913_2406805_011.tif'

# Define metadata for the raster data
metadata = {
    'driver': 'GTiff',
    'dtype': 'float32',
    'nodata': None,
    'width': emit_ds.sizes['longitude'],
    'height': emit_ds.sizes['latitude'],
    'count': 3,  # Number of bands (RGB)
    'compress': 'lzw',  # Compression method (Lempel-Ziv-Welch)
    'crs': 'EPSG:4326', # WGS 84 ((World Geodetic System 1984))
    'transform': emit_ds['reflectance'].rio.transform()
}

# Transpose the array to match the expected dimensions for writing
reflectance_array = np.transpose(reflectance_array, axes=(2, 0, 1))  # Move the band dimension to the first axis

# Write the array to a raster file
with rasterio.open(output_path, 'w', **metadata) as dst:
    dst.write(reflectance_array)

print(f"True color image saved to {output_path}")

True color image saved to F:\emit\EMIT\March_s3\EMIT_L2A_RFL_001_20240308T070913_2406805_011.tif
